## Install

In [ ]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth
!pip install gdown

import torch
import gdown
import pandas as pd
from datasets import Dataset
from unsloth import FastLanguageModel
from transformers import TrainingArguments, DataCollatorForSeq2Seq, TextStreamer
from trl import SFTTrainer
from sklearn.model_selection import train_test_split


## Connect drive

In [7]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [11]:
import os

folder_path = "/content/drive/MyDrive/vinallama_poem_model"
if os.path.exists(folder_path):
    print("Thư mục tồn tại.")
else:
    print("Thư mục chưa tồn tại.")


Thư mục tồn tại.


## Load model and tokenizer

In [ ]:
from unsloth import FastLanguageModel

save_path = "/content/drive/MyDrive/vinallama_poem_model"

max_seq_length = 4096
dtype = "float16"
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=save_path,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

FastLanguageModel.for_inference(model)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


## Predict function

In [114]:
import textwrap

def generate_poem(prompt, max_new_tokens=200):
    text = f"""<|im_start|>system
Bạn là một AI thi sĩ chuyên sáng tác thơ lục bát bằng tiếng Việt.
Hãy thể hiện cảm xúc tinh tế, sử dụng ngôn từ đẹp, và tuân thủ nghiêm ngặt các quy tắc sau:
- Bài thơ có nhiều cặp dòng: một dòng 6 chữ, sau đó là một dòng 8 chữ.
- Mỗi dòng phải xuống dòng rõ ràng.
- Dòng 6 chữ có đúng 6 từ, dòng 8 chữ có đúng 8 từ.
- Không được viết sai nhịp, không viết quá, viết thiếu số chữ.
- Vần luật phải tự nhiên, đúng phong cách thơ ca truyền thống Việt Nam.<|im_end|>
<|im_start|>user
Hãy sáng tác một bài thơ lục bát về chủ đề '{prompt}'<|im_end|>

<|im_start|>assistant
"""

    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_seq_length
    )

    inputs = {key: value.to(device) for key, value in inputs.items()}

    output = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.8,
        top_k=50,
        top_p=0.9,
        repetition_penalty=1.1
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_poem = generated_text.replace(text, "").strip()
    return generated_poem


In [115]:
theme = "vầng trăng"

In [116]:
generated_poem = generate_poem(theme)

In [117]:
print(generated_poem)

<|im_start|> system
Bạn là một AI thi sĩ chuyên sáng tác thơ lục bát bằng tiếng Việt.
Hãy thể hiện cảm xúc tinh tế, sử dụng ngôn từ đẹp, và tuân thủ nghiêm ngặt các quy tắc sau:
- Bài thơ có nhiều cặp dòng: một dòng 6 chữ, sau đó là một dòng 8 chữ.
- Mỗi dòng phải xuống dòng rõ ràng.
- Dòng 6 chữ có đúng 6 từ, dòng 8 chữ có đúng 8 từ.
- Không được viết sai nhịp, không viết quá, viết thiếu số chữ.
- Vần luật phải tự nhiên, đúng phong cách thơ ca truyền thống Việt Nam. 
<|im_start|> user
Hãy sáng tác một bài thơ lục bát về chủ đề 'vầng trăng' 

<|im_start|> assistant
trăng ơi sao cứ lặng thinh
để em nhung nhớ đêm trường cô đơn
mây buồn phủ kín bầu trời
giọt sương lạnh lẽo ướt rơi bên thềm


## fastapi and ngronk

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn


In [88]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import torch

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class Prompt(BaseModel):
    title: str


In [97]:
@app.post("/generate_poem")
def gen_poem(prompt: Prompt):
    generated_poem = generate_poem(prompt)
    generated_poem = generated_poem.split("assistant")[-1].strip()
    return {"poem": generated_poem}


In [111]:

import nest_asyncio
import uvicorn
from pyngrok import conf, ngrok

nest_asyncio.apply()
conf.get_default().auth_token = "30QBZhUD0nbtwQptJyO8Vk1JKEW_5CtJidGwFCNFDN1B2YUKt"

public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

uvicorn.run(app, host="0.0.0.0", port=8000)


Public URL: NgrokTunnel: "https://702899f907d7.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [752]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [752]
